In [574]:
from pandas import DataFrame, read_csv, to_numeric
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from bigml.api import BigML
import kaggle

%matplotlib inline

In [ ]:
validation = True   #Do we split the data?
compression = True  #Do we compress the csv files?
send_to_BigML = True
build_model = True

version='v1.3'

In [ ]:
prediction_file='./gsmc_predictions-{}.csv'.format(version)

## Open prediction file

In [ ]:
prediction=read_csv(prediction_file,index_col=0)

## Send prediction to Kaggle

In [ ]:
kaggle_prediction=DataFrame(index=prediction.index)

kaggle_prediction['Id']=prediction['Id']
kaggle_prediction['Probability']=prediction['1 probability']
kaggle_prediction_file="kaggleprediction.csv"
kaggle_prediction.to_csv(kaggle_prediction_file,index=False)
kaggle.api.competition_submit(kaggle_prediction_file, "Ensemble-" + str(version), "GiveMeSomeCredit")

In [ ]:
kaggle.api.competitions_submissions_list("GiveMeSomeCredit")[0]['publicScore']